In [1]:
import networkx as nx
import HW1
import re
from pathlib import Path
import pandas as pd
import os
import numpy as np

In [ ]:
#EDA number of classes/app, most common library used, graph-degree related statistics
#basically do all those but compare between categories

In [ ]:
HW1.main()

In [2]:
def getAllSmaliFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries    
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getAllSmaliFiles(fullPath)
        else:
            allFiles.append(fullPath)
    allFiles = [i for i in allFiles if i.split(".")[-1] == "smali"] 
    return allFiles 

In [3]:
smalis = getAllSmaliFiles("Apps")

In [5]:
#total number of smali files for 423 apps
len(smalis)

2153825

In [4]:
#key = app name, value = dictionary: key: api_call : name of api : # of occurance/ lib_call: name of lib: # of occurance
app_features = {}
#key = genre, value = list of apps that belong in that genre
genre_apps = {}
total_api_calls = 0
for i in smalis:
    temp = i.split("/")
    #temp[2]/curr is the app name
    curr = temp[2]
    #temp[1] is the genre
    if temp[1] not in genre_apps:
        genre_apps[temp[1]] = []
    if curr not in app_features:
        app_features[curr] = {'api_calls': {}, 'lib_calls': {}}
    txt = Path(i).read_text(errors = 'ignore')
    lib_calls = re.findall(r'L.+->', txt) 
    api_calls = re.findall(r'L.+->.+\(', txt)
    total_api_calls += len(api_calls)
    #iterate through api calls
    for j in api_calls:
        if j not in app_features[curr]['api_calls']:
            app_features[curr]['api_calls'][j] = 0
        app_features[curr]['api_calls'][j]+=1
    #iterate through library calls
    for k in lib_calls:
        if k not in app_features[curr]['lib_calls']:
            app_features[curr]['lib_calls'][k] = 0
        app_features[curr]['lib_calls'][k]+=1
    if curr not in genre_apps[temp[1]]:
        genre_apps[temp[1]].append(curr)
total_api_calls #with duplicates

47075524

In [7]:
len(genre_apps['productivity'])

211

In [7]:
most_common = {}
for genre in genre_apps:
    most_common[genre] = {'api': ["", 0], 'lib': ["", 0]}
    for app in genre_apps[genre]:
        api_count = list(app_features[app]["api_calls"].items())
        #print(app)
        lib_count = list(app_features[app]["lib_calls"].items())
        
        mode_api = max(api_count, key=lambda x:x[1])
        mode_lib = max(lib_count, key=lambda x:x[1])
        
        curr_mode_api = most_common[genre]["api"]
        if mode_api[0] == curr_mode_api[0]:
            curr_mode_api[1] += mode_api[1]
        else:
            if mode_api[1] > curr_mode_api[1]:
                curr_mode_api[0] = mode_api[0]
                curr_mode_api[1] = mode_api[1]
                
        curr_mode_lib = most_common[genre]["lib"]
        if mode_lib[0] == curr_mode_lib[0]:
            curr_mode_lib[1] += mode_lib[1]
        else:
            if mode_lib[1] > curr_mode_lib[1]:
                curr_mode_lib[0] = mode_lib[0]
                curr_mode_lib[1] = mode_lib[1]
#most_common

In [8]:
most_common['entertainment']["api"]

['Ljava/lang/StringBuilder;->append(', 1008310]

In [9]:
most_common['entertainment']["lib"]

['Lcom/google/android/gms/internal/zzai;->', 1165710]

In [10]:
print(most_common['productivity']["api"])
most_common['productivity']["lib"]

['Ljava/lang/StringBuilder;->append(', 1095257]


['Ljava/lang/StringBuilder;->', 1211707]

In [11]:
import numpy as np

In [12]:
#encode the apis
common_api = [] #could've done this in the app-features section
common_lib = [] #ok this actually takes a while definitely do this in the app_features section
x = []
labels = []
for genre in genre_apps:
    
    for i in genre_apps[genre]:
        labels.append(genre)
        api_count = list(app_features[i]["api_calls"].items())
        lib_count = list(app_features[i]["lib_calls"].items())
        
        mode_api = max(api_count, key=lambda x:x[1])[0]
        mode_lib = max(lib_count, key=lambda x:x[1])[0]
    
        if mode_api not in common_api:
            common_api.append(mode_api)
        if mode_lib not in common_lib:
            common_lib.append(mode_lib)
        api_index = common_api.index(mode_api)
        lib_index = common_lib.index(mode_lib)
        degree = len(np.unique(app_features[i]["api_calls"]))
        x.append([api_index, lib_index, degree])

print(len(x))
print(len(labels))
print(len(common_api))
print(len(common_lib))

423
423
10
59


In [13]:
labels_test = labels[:15] + labels[-15:]
labels_test

['entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity',
 'productivity']

In [14]:
x_test = x[:15] + x[-15:]
len(x_test)

30

In [15]:
x_train = x[15:-15]
print(len(x_train))
labels_train = labels[15:-15]
print(len(labels_train))

393
393


In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train, labels_train)
dtc_preds = dtc.predict(x_train)
print("Train Accuracy: ", metrics.accuracy_score(labels_train, dtc_preds))
dtc_preds = dtc.predict(x_test)
print("Accuracy: ", metrics.accuracy_score(labels_test, dtc_preds))

Train Accuracy:  0.7938931297709924
Accuracy:  0.6666666666666666


In [18]:
rfc = RandomForestClassifier()
rfc.fit(x_train, labels_train)

rfc_preds = rfc.predict(x_train)
print("TrainAccuracy: ", metrics.accuracy_score(labels_train, rfc_preds))

rfc_preds = rfc.predict(x_test)
print("Accuracy: ", metrics.accuracy_score(labels_test, rfc_preds))

TrainAccuracy:  0.7913486005089059
Accuracy:  0.6666666666666666


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [19]:
gbc = GradientBoostingClassifier()
gbc.fit(x_train, labels_train)
gbc_preds = gbc.predict(x_train)
print("TrainAccuracy", metrics.accuracy_score(labels_train, gbc_preds))

gbc_preds = gbc.predict(x_test)
print("Accuracy", metrics.accuracy_score(labels_test, gbc_preds))

TrainAccuracy 0.7938931297709924
Accuracy 0.6666666666666666
